In [1]:
import mlflow
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing
import tensorflow_addons as tfa

from tensorflow.keras.callbacks import EarlyStopping

from tabtransformertf.models.fttransformer import FTTransformerEncoder, FTTransformer
from tabtransformertf.utils.preprocessing import df_to_dataset

import catboost as cb
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError

d:\softs\conda\envs\github\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
d:\softs\conda\envs\github\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sur

In [ ]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db --port 5000

In [2]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams.update({'font.size': 15})

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("income")

<Experiment: artifact_location='file:///d:/study/github/Learning-MLFlow/notebooks/mlruns/1', creation_time=1712957674467, experiment_id='1', last_update_time=1712957674467, lifecycle_stage='active', name='income', tags={}>

## Data

In [19]:
dset = fetch_california_housing()
data = dset['data']
y = dset['target']
LABEL = dset['target_names'][0]

NUMERIC_FEATURES = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Longitude', 'Latitude']
FEATURES = NUMERIC_FEATURES

data = pd.DataFrame(data, columns=dset['feature_names'])
data[LABEL] = y

data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [20]:
train_data, test_data = train_test_split(data, test_size=0.2)
print(f"Train dataset shape: {train_data.shape}")
print(f"Test dataset shape: {test_data.shape}")

Train dataset shape: (16512, 9)
Test dataset shape: (4128, 9)


### split dataset

In [21]:
X_train, X_val = train_test_split(train_data, test_size=0.2)

sc = StandardScaler()
X_train.loc[:, NUMERIC_FEATURES] = sc.fit_transform(X_train[NUMERIC_FEATURES])
X_val.loc[:, NUMERIC_FEATURES] = sc.transform(X_val[NUMERIC_FEATURES])
test_data.loc[:, NUMERIC_FEATURES] = sc.transform(test_data[NUMERIC_FEATURES])

## Baseline

In [7]:
mlflow.sklearn.autolog(disable=True)

with mlflow.start_run(run_name='rf_baseline'):
    params = {
        "n_estimators": 100,
        "max_depth": 20
    }

    mlflow.set_tag("model_name", "RF")
    mlflow.log_params(params)

    rf = RandomForestRegressor(n_estimators=params["n_estimators"], max_depth=params["max_depth"])
    rf.fit(X_train[FEATURES], X_train[LABEL])

    rf_preds = rf.predict(test_data[FEATURES])
    rf_rms = mean_squared_error(test_data[LABEL], rf_preds, squared=False)

    mlflow.log_metric("test_rmse", rf_rms)
    mlflow.sklearn.log_model(rf, "sk_models")

d:\softs\conda\envs\github\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
d:\softs\conda\envs\github\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## CatBoost

In [8]:
catb_train_dataset = cb.Pool(X_train[FEATURES], X_train[LABEL]) 
catb_val_dataset = cb.Pool(X_val[FEATURES], X_val[LABEL]) 
catb_test_dataset = cb.Pool(test_data[FEATURES], test_data[LABEL])

In [9]:
with mlflow.start_run(run_name="catboost"):
    mlflow.set_tag("model_name", "CatBoost")
    catb = cb.CatBoostRegressor()
    catb.fit(catb_train_dataset, eval_set=catb_val_dataset, early_stopping_rounds=50)
    catb_preds = catb.predict(catb_test_dataset)
    catb_rms = mean_squared_error(test_data[LABEL], catb_preds, squared=False)

    mlflow.log_metric("test_rmse", catb_rms)
    mlflow.catboost.log_model(catb, "cb_models")

Learning rate set to 0.076361
0:	learn: 1.1172394	test: 1.0757106	best: 1.0757106 (0)	total: 179ms	remaining: 2m 58s
1:	learn: 1.0710392	test: 1.0318532	best: 1.0318532 (1)	total: 181ms	remaining: 1m 30s
2:	learn: 1.0316702	test: 0.9939944	best: 0.9939944 (2)	total: 183ms	remaining: 1m
3:	learn: 0.9935661	test: 0.9576509	best: 0.9576509 (3)	total: 185ms	remaining: 46.1s
4:	learn: 0.9601693	test: 0.9259167	best: 0.9259167 (4)	total: 187ms	remaining: 37.3s
5:	learn: 0.9288534	test: 0.8962684	best: 0.8962684 (5)	total: 190ms	remaining: 31.4s
6:	learn: 0.9014473	test: 0.8697088	best: 0.8697088 (6)	total: 192ms	remaining: 27.2s
7:	learn: 0.8754579	test: 0.8465154	best: 0.8465154 (7)	total: 194ms	remaining: 24s
8:	learn: 0.8534922	test: 0.8259699	best: 0.8259699 (8)	total: 196ms	remaining: 21.5s
9:	learn: 0.8314109	test: 0.8052366	best: 0.8052366 (9)	total: 198ms	remaining: 19.6s
10:	learn: 0.8122064	test: 0.7873392	best: 0.7873392 (10)	total: 200ms	remaining: 18s
11:	learn: 0.7938715	test: 

d:\softs\conda\envs\github\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [10]:
def build_mlp(params):
    mlp = Sequential([
        Dense(params["layer1_size"], activation=params['activation']),
        Dropout(params['dropout_rate']),
        Dense(params["layer2_size"], activation=params['activation']),
        Dropout(params['dropout_rate']),
        Dense(params["layer3_size"], activation=params['activation']),
        Dense(1, activation='relu')
    ])
    return mlp

def train_mlp(mlp, train_params, train_dataset, val_dataset):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=train_params["learning_rate"],
        weight_decay=train_params["weight_decay"],
    )
    mlp.compile(
        optimizer=optimizer,
        loss=MeanSquaredError(name="mse"),
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")]
    )

    early = EarlyStopping(
        monitor="val_loss",
        mode="min",
        patience=train_params["early_stop_patience"],
        restore_best_weights=True,
    )
    callback_list = [early]

    hist = mlp.fit(
        train_dataset,
        epochs=train_params["num_epochs"],
        validation_data=val_dataset,
        callbacks=callback_list,
    )
    return mlp


def mlp_mlflow_run(
    name,
    mlp_params,
    train_params,
    train_dataset,
    val_dataset,
    test_dataset,
    y_test,
):
    with mlflow.start_run(run_name=name):
        mlflow.log_params(mlp_params)
        mlflow.log_params(train_params)
        mlflow.set_tag("model_name", "MLP")
        mlp = build_mlp(mlp_params)
        mlp = train_mlp(mlp, train_params, train_dataset, val_dataset)
        test_preds = mlp.predict(test_dataset)
        test_rms = mean_squared_error(
            y_test, test_preds.ravel(), squared=False
        )
        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(mlp, "tf_models")

In [11]:
mlp_train_ds = tf.data.Dataset.from_tensor_slices((X_train[FEATURES], X_train[LABEL])).batch(512).shuffle(512*4).prefetch(512)
mlp_val_ds = tf.data.Dataset.from_tensor_slices((X_val[FEATURES], X_val[LABEL])).batch(512).shuffle(512*4).prefetch(512)
mlp_test_ds = tf.data.Dataset.from_tensor_slices(test_data[FEATURES]).batch(512).prefetch(512)

mlp_params = {
    "layer1_size": 512,
    "layer2_size": 128,
    "layer3_size": 64,
    "dropout_rate": 0.3,
    "activation": 'relu'

}
train_params = dict(
    learning_rate=0.001, weight_decay=0.00001, early_stop_patience=10, num_epochs=1000
)

mlp_mlflow_run(
    "mlp_base",
    mlp_params,
    train_params,
    mlp_train_ds,
    mlp_val_ds,
    mlp_test_ds,
    test_data[LABEL],
)

Epoch 1/1000
26/26 [==============================] - 1s 9ms/step - loss: 2.4125 - rmse: 1.5532 - val_loss: 0.8411 - val_rmse: 0.9171
Epoch 2/1000
26/26 [==============================] - 0s 5ms/step - loss: 0.8666 - rmse: 0.9309 - val_loss: 0.6140 - val_rmse: 0.7836
Epoch 3/1000
26/26 [==============================] - 0s 6ms/step - loss: 0.6529 - rmse: 0.8080 - val_loss: 0.4781 - val_rmse: 0.6914
Epoch 4/1000
26/26 [==============================] - 0s 5ms/step - loss: 0.5474 - rmse: 0.7399 - val_loss: 0.4213 - val_rmse: 0.6491
Epoch 5/1000
26/26 [==============================] - 0s 5ms/step - loss: 0.5026 - rmse: 0.7090 - val_loss: 0.3905 - val_rmse: 0.6249
Epoch 6/1000
26/26 [==============================] - 0s 5ms/step - loss: 0.4708 - rmse: 0.6861 - val_loss: 0.3742 - val_rmse: 0.6117
Epoch 7/1000
26/26 [==============================] - 0s 6ms/step - loss: 0.4535 - rmse: 0.6734 - val_loss: 0.3592 - val_rmse: 0.5993
Epoch 8/1000
26/26 [==============================] - 0s 6ms/s

d:\softs\conda\envs\github\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/04/13 10:42:31 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\grego\AppData\Local\Temp\tmplujm32sb\model\data\model\assets


2024/04/13 10:42:36 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\grego\AppData\Local\Temp\tmplujm32sb\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.0']. Set logging level to DEBUG to see the full traceback. 


In [17]:
mlp_mlflow_run

<function __main__.mlp_mlflow_run(name, mlp_params, train_params, train_dataset, val_dataset, test_dataset, y_test)>

In [22]:
train_dataset = df_to_dataset(X_train[FEATURES + [LABEL]], LABEL, shuffle=True)
val_dataset = df_to_dataset(X_val[FEATURES + [LABEL]], LABEL, shuffle=False)  # No shuffle
test_dataset = df_to_dataset(test_data[FEATURES], shuffle=False) # No target, no shuffle

ValueError: Multi-dimensional indexing (e.g. `obj[:, None]`) is no longer supported. Convert to a numpy array before indexing instead.

In [29]:
def modified_df_to_dataset(dataframe, target=None, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    if target:
        labels = dataframe.pop(target)
    else:
        labels = None
    dataset = {}
    for key, value in dataframe.items():
        dataset[key] = value.values[:, tf.newaxis]
    dataset = tf.data.Dataset.from_tensor_slices((dataset, labels))
    if shuffle and labels is not None:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    dataset = dataset.batch(batch_size)
    return dataset


train_dataset = modified_df_to_dataset(X_train[FEATURES + [LABEL]], LABEL, shuffle=True)
val_dataset = modified_df_to_dataset(X_val[FEATURES + [LABEL]], LABEL, shuffle=False)
test_dataset = modified_df_to_dataset(test_data[FEATURES], target=None, shuffle=False) # No target, no shuffle

In [30]:
def build_fttransformer(
    params_to_log, params_to_skip, out_dim=1, out_activation="relu"
):
    # Define encoder
    ft_encoder = FTTransformerEncoder(
        **params_to_log,
        **params_to_skip
    )
    # Add prediction head to the encoder
    ft_transformer = FTTransformer(
        encoder=ft_encoder,
        out_dim=out_dim,
        out_activation=out_activation,
    )

    return ft_transformer


def train_model(model, train_params, train_dataset, val_dataset):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=train_params["learning_rate"],
        weight_decay=train_params["weight_decay"],
    )

    model.compile(
        optimizer=optimizer,
        loss={
            "output": tf.keras.losses.MeanSquaredError(name="mse"),
            "importances": None,
        },
        metrics={
            "output": [tf.keras.metrics.RootMeanSquaredError(name="rmse")],
            "importances": None,
        },
    )

    early = EarlyStopping(
        monitor="val_output_loss",
        mode="min",
        patience=train_params["early_stop_patience"],
        restore_best_weights=True,
    )
    callback_list = [early]

    hist = model.fit(
        train_dataset,
        epochs=train_params["num_epochs"],
        validation_data=val_dataset,
        callbacks=callback_list,
    )
    return model

In [31]:
mlflow.tensorflow.autolog(disable=True)


def fttransformer_mlflow_run(
    name,
    encoder_params,
    train_params,
    params_to_skip,
    train_dataset,
    val_dataset,
    test_dataset,
    y_test,
):
    with mlflow.start_run(run_name=name):
        mlflow.set_tag("model_name", "FTTransformer")
        # Log the params
        mlflow.log_params(encoder_params)
        mlflow.log_params(train_params)
        # Build and train
        ft_transformer = build_fttransformer(
            encoder_params,
            params_to_skip,
            out_dim=1,
            out_activation="relu",
        )
        ft_transformer = train_model(
            ft_transformer, train_params, train_dataset, val_dataset
        )
        # Evaluate
        test_preds = ft_transformer.predict(test_dataset)
        test_rms = mean_squared_error(
            y_test, test_preds["output"].ravel(), squared=False
        )
        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(ft_transformer, "tf_models")

In [32]:
train_params = dict(
    learning_rate=0.001, weight_decay=0.00001, early_stop_patience=10, num_epochs=1000
)

params_to_skip = dict(
    numerical_data=X_train[NUMERIC_FEATURES].values,
    categorical_data=None,
    y=X_train[LABEL].values
)

### Linear Embedding

In [33]:
linear_embeddings_params = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type="linear",
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

fttransformer_mlflow_run(
    name='linear',
    encoder_params=linear_embeddings_params,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
413/413 [==============================] - 9s 15ms/step - loss: 0.7383 - output_loss: 0.7383 - output_rmse: 0.8593 - val_loss: 0.4570 - val_output_loss: 0.4570 - val_output_rmse: 0.6760
Epoch 2/1000
413/413 [==============================] - 6s 14ms/step - loss: 0.4814 - output_loss: 0.4814 - output_rmse: 0.6938 - val_loss: 0.3986 - val_output_loss: 0.3986 - val_output_rmse: 0.6313
Epoch 3/1000
413/413 [==============================] - 6s 14ms/step - loss: 0.4374 - output_loss: 0.4374 - output_rmse: 0.6613 - val_loss: 0.3842 - val_output_loss: 0.3842 - val_output_rmse: 0.6198
Epoch 4/1000
413/413 [==============================] - 6s 14ms/step - loss: 0.4071 - output_loss: 0.4071 - output_rmse: 0.6381 - val_loss: 0.3285 - val_output_loss: 0.3285 - val_output_rmse: 0.5731
Epoch 5/1000
413/413 [==============================] - 6s 14ms/step - loss: 0.3939 - output_loss: 0.3939 - output_rmse: 0.6276 - val_loss: 0.3796 - val_output_loss: 0.3796 - val_output_rmse: 0.6161
Epoch

d:\softs\conda\envs\github\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/04/13 11:36:53 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\grego\AppData\Local\Temp\tmp28xl63yb\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\grego\AppData\Local\Temp\tmp28xl63yb\model\data\model\assets
2024/04/13 11:37:05 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\grego\AppData\Local\Temp\tmp28xl63yb\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.0']. Set logging level to DEBUG to see the full traceback. 


### Periodic

In [34]:
periodic_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='periodic',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

fttransformer_mlflow_run(
    name='periodic',
    encoder_params=periodic_params_to_log,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
413/413 [==============================] - 12s 20ms/step - loss: 0.5927 - output_loss: 0.5927 - output_rmse: 0.7699 - val_loss: 0.3711 - val_output_loss: 0.3711 - val_output_rmse: 0.6092
Epoch 2/1000
413/413 [==============================] - 8s 20ms/step - loss: 0.3773 - output_loss: 0.3773 - output_rmse: 0.6142 - val_loss: 0.3422 - val_output_loss: 0.3422 - val_output_rmse: 0.5850
Epoch 3/1000
413/413 [==============================] - 8s 19ms/step - loss: 0.3291 - output_loss: 0.3291 - output_rmse: 0.5737 - val_loss: 0.2814 - val_output_loss: 0.2814 - val_output_rmse: 0.5305
Epoch 4/1000
413/413 [==============================] - 8s 19ms/step - loss: 0.3032 - output_loss: 0.3032 - output_rmse: 0.5507 - val_loss: 0.3216 - val_output_loss: 0.3216 - val_output_rmse: 0.5671
Epoch 5/1000
413/413 [==============================] - 8s 19ms/step - loss: 0.2883 - output_loss: 0.2883 - output_rmse: 0.5369 - val_loss: 0.2908 - val_output_loss: 0.2908 - val_output_rmse: 0.5393
Epoc

d:\softs\conda\envs\github\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/04/13 11:52:12 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\grego\AppData\Local\Temp\tmpe2o60lyf\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\grego\AppData\Local\Temp\tmpe2o60lyf\model\data\model\assets
2024/04/13 11:52:24 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\grego\AppData\Local\Temp\tmpe2o60lyf\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.0']. Set logging level to DEBUG to see the full traceback. 


### PLE - Quantile Binning

In [38]:
pleq_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='ple', # ple or pleq is the same for the encoder params
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

pleq_params_to_skip = params_to_skip.copy()
pleq_params_to_skip['y'] = None

fttransformer_mlflow_run(
    name='ple_quantile',
    encoder_params=pleq_params_to_log,
    train_params=train_params,
    params_to_skip=pleq_params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
413/413 [==============================] - 203s 244ms/step - loss: 0.5303 - output_loss: 0.5303 - output_rmse: 0.7282 - val_loss: 0.3621 - val_output_loss: 0.3621 - val_output_rmse: 0.6018
Epoch 2/1000
413/413 [==============================] - 46s 112ms/step - loss: 0.3444 - output_loss: 0.3444 - output_rmse: 0.5869 - val_loss: 0.2604 - val_output_loss: 0.2604 - val_output_rmse: 0.5103
Epoch 3/1000
413/413 [==============================] - 47s 113ms/step - loss: 0.3042 - output_loss: 0.3042 - output_rmse: 0.5516 - val_loss: 0.3020 - val_output_loss: 0.3020 - val_output_rmse: 0.5496
Epoch 4/1000
413/413 [==============================] - 46s 111ms/step - loss: 0.2829 - output_loss: 0.2829 - output_rmse: 0.5319 - val_loss: 0.2471 - val_output_loss: 0.2471 - val_output_rmse: 0.4971
Epoch 5/1000
413/413 [==============================] - 45s 109ms/step - loss: 0.2725 - output_loss: 0.2725 - output_rmse: 0.5220 - val_loss: 0.2444 - val_output_loss: 0.2444 - val_output_rmse: 0

d:\softs\conda\envs\github\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/04/13 12:31:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\grego\AppData\Local\Temp\tmp8gzdyemi\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\grego\AppData\Local\Temp\tmp8gzdyemi\model\data\model\assets
2024/04/13 12:38:13 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\grego\AppData\Local\Temp\tmp8gzdyemi\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.0']. Set logging level to DEBUG to see the full traceback. 


### PLE - Target Binning

In [39]:
plet_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='ple',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
    task='regression',
    ple_tree_params = {
        "min_samples_leaf": 20,
    }
)


fttransformer_mlflow_run(
    name='ple_target',
    encoder_params=plet_params_to_log,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
413/413 [==============================] - 347s 339ms/step - loss: 0.5047 - output_loss: 0.5047 - output_rmse: 0.7104 - val_loss: 0.3589 - val_output_loss: 0.3589 - val_output_rmse: 0.5991
Epoch 2/1000
413/413 [==============================] - 93s 226ms/step - loss: 0.3401 - output_loss: 0.3401 - output_rmse: 0.5831 - val_loss: 0.3069 - val_output_loss: 0.3069 - val_output_rmse: 0.5540
Epoch 3/1000
413/413 [==============================] - 94s 227ms/step - loss: 0.3053 - output_loss: 0.3053 - output_rmse: 0.5526 - val_loss: 0.3140 - val_output_loss: 0.3140 - val_output_rmse: 0.5604
Epoch 4/1000
413/413 [==============================] - 93s 226ms/step - loss: 0.2965 - output_loss: 0.2965 - output_rmse: 0.5445 - val_loss: 0.3079 - val_output_loss: 0.3079 - val_output_rmse: 0.5549
Epoch 5/1000
413/413 [==============================] - 93s 226ms/step - loss: 0.2746 - output_loss: 0.2746 - output_rmse: 0.5241 - val_loss: 0.2959 - val_output_loss: 0.2959 - val_output_rmse: 0

d:\softs\conda\envs\github\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/04/13 13:53:19 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\grego\AppData\Local\Temp\tmpflep0dh2\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\grego\AppData\Local\Temp\tmpflep0dh2\model\data\model\assets
2024/04/13 14:01:35 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\grego\AppData\Local\Temp\tmpflep0dh2\model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.0']. Set logging level to DEBUG to see the full traceback. 
